# Initialization

Change to `valuenet` directory and add `src` path to `PYTHONPATH`.

In [ ]:
%cd /home/ec2-user/SageMaker/valuenetforked

In [ ]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/valuenetforked/src')

Read environment

In [ ]:
NER_API_SECRET=%env NER_API_SECRET
API_KEY=%env API_KEY
DB_USER=%env DB_USER
DB_PW=%env DB_PW
DB_HOST=%env DB_HOST
DB_PORT=%env DB_PORT
DB_SCHEMA="public"

# Prepare & Preprocess Data

We follow the [user manual from Valuenet](https://github.com/brunnurs/valuenet).

## Add your custom data

An example of custom data preparation can be found in statbot repository: [generate_sql_statments_and_questions.ipynb](https://github.com/statistikZH/statbot/blob/main/hackathon_hackzurich/generate_sql_statments_and_questions.ipynb). In this repository, random values are taken from the hack_zurich database and fed through a template to generate questions and queries.
We then convert these questions and queries in the required format and save them as 
- statbot/hackathon_hackzurich/handmade_data_dev.json
- statbot/hackathon_hackzurich/handmade_data_train.json

If you wish to then preprocess these generated data, copy the handmade_data_xxx.json from statbot to valuenet:
- valuenet/data/hack_zurich/handmade_training_data/handmade_data_dev.json'
- valuenet/data/hack_zurich/handmade_training_data/handmade_data_train.json'

You are now ready to preprocess the dataset by running the following steps.

**Note**: If you decide to create your own query template, make sure to be very careful of the syntax and stay close to the one in the example notebook as valuenet codebase is quite sensible to the queries. For instance, 
- as it tokenize the SQL queries based on spaces, make sure to always add spaces everywhere (T1.year=2006 will error and sould be replaced with T1.year = 2006)
- always put the keyword 'AS' when using shortcut names

If you have errors in the next step, it is probably because of one of this reason. If error persist, check the effect of the function `tokenize()` in `valuenet/src/spider/test_suite_eval/process_sql.py` and adapt the codebase.

## Transform into Spider representation

In [ ]:
%run src/tools/training_data_builder/training_data_builder.py --data hack_zurich

You will now find your custom data in the two files [data/hack_zurich/original/train.json](data/hack_zurich/original/train.json) and [data/hack_zurich/original/dev.json](data/hack_zurich/original/dev.json).

## Extract Value Candidates using Named Entity Recognition

In [ ]:
%run src/named_entity_recognition/api_ner/extract_values.py --data_path=data/hack_zurich/original/train.json --output_path=data/hack_zurich/ner_train.json --ner_api_secret={NER_API_SECRET}

In [ ]:
%run src/named_entity_recognition/api_ner/extract_values.py --data_path=data/hack_zurich/original/dev.json --output_path=data/hack_zurich/ner_dev.json --ner_api_secret={NER_API_SECRET}

## Extract the ground truth values from the SQL query

In [ ]:
%run src/tools/get_values_from_sql.py --data_path data/hack_zurich/original/train.json --table_path data/hack_zurich/original/tables.json --ner_path data/hack_zurich/ner_train.json
%run src/tools/get_values_from_sql.py --data_path data/hack_zurich/original/dev.json --table_path data/hack_zurich/original/tables.json --ner_path data/hack_zurich/ner_dev.json

This last script doesn't create a new file, but adds the ground truth values to the *ner_dev.json* and *ner_train.json* files, see the new attribute *values*:

```json
    "values": [
      "Wetzikon",
      "2016"
    ]
```

## Pre-processing

In [ ]:
%run src/preprocessing/pre_process.py --data_path=data/hack_zurich/original/train.json --ner_data_path=data/hack_zurich/ner_train.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_train.json --database_host={DB_HOST} --database_port={DB_PORT} --database_user={DB_USER} --database_password={DB_PW} --database_schema={DB_SCHEMA}

In [ ]:
%run src/preprocessing/pre_process.py --data_path=data/hack_zurich/original/dev.json --ner_data_path=data/hack_zurich/ner_dev.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_dev.json --database_host={DB_HOST} --database_port={DB_PORT} --database_user={DB_USER} --database_password={DB_PW} --database_schema={DB_SCHEMA}

## Modelling JOINs and SQL-to-SemQL

We start by modeling some JOINs as filters (minor importance, has most probably no effect on your data - you might skip it)

In [ ]:
%run src/preprocessing/model_joins_as_filter.py --data_path=data/hack_zurich/preprocessed_train.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_with_joins_train.json 

In [ ]:
%run src/preprocessing/model_joins_as_filter.py --data_path=data/hack_zurich/preprocessed_dev.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_with_joins_dev.json 

And then transform SQL to SemQL

In [ ]:
%run src/preprocessing/sql2SemQL.py --data_path data/hack_zurich/preprocessed_with_joins_train.json --table_path data/hack_zurich/original/tables.json --output data/hack_zurich/train.json

In [ ]:
%run src/preprocessing/sql2SemQL.py --data_path data/hack_zurich/preprocessed_with_joins_dev.json --table_path data/hack_zurich/original/tables.json --output data/hack_zurich/dev.json 

You should now be able to **train** with the [train-01.ipynb](https://github.com/hack-with-admin-ch/aws-sagemaker-notebook-valuenet/blob/main/train-01.ipynb).